A CNN model can be used in a hybrid model with an LSTM backend where the CNN is used to interpret subsequences of input that together are provided as a sequence to an LSTM model to interpret. This hybrid model is called a CNN-LSTM.

The first step is to split the input sequences into subsequences that can be processed by the CNN model. For example, we can first split our univariate time series data into input/output samples with four steps as input and one as output. Each sample can then be split into two sub-samples, each with two time steps. The CNN can interpret each subsequence of two time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [ ]:
10

In [1]:
import numpy as np

In [2]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [3]:
# choose a number of time steps
n_steps = 4

# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])


[10 20 30 40] 50
[20 30 40 50] 60
[30 40 50 60] 70
[40 50 60 70] 80
[50 60 70 80] 90


In [5]:
X

array([[10, 20, 30, 40],
       [20, 30, 40, 50],
       [30, 40, 50, 60],
       [40, 50, 60, 70],
       [50, 60, 70, 80]])


[samples, subsequences, timesteps, features]

In [6]:
n_seq=2
n_steps=2
n_features=1
X_reshaped=X.reshape(X.shape[0],n_seq,n_steps,n_features)

In [9]:
print(X_reshaped)
X_reshaped.shape

[[[[10]
   [20]]

  [[30]
   [40]]]


 [[[20]
   [30]]

  [[40]
   [50]]]


 [[[30]
   [40]]

  [[50]
   [60]]]


 [[[40]
   [50]]

  [[60]
   [70]]]


 [[[50]
   [60]]

  [[70]
   [80]]]]


(5, 2, 2, 1)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D,Flatten,Dense


In [13]:
model=Sequential()
model.add(TimeDistributed(Conv1D(filters=64,kernel_size=1,activation="relu",),input_shape=(None,n_steps,n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(50,activation='relu'))
model.add(Dense(1))
model.summary()



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 2, 64)       128       
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1, 64)       0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 64)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                23000     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 23,179
Trainable params: 23,179
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])


In [27]:
history=model.fit(X_reshaped,y,epochs=100,verbose=1)


Epoch 1/100
5/5 [==============================] - 0s 1ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 0s 2ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 3/100
5/5 [==============================] - 0s 1ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 4/100
5/5 [==============================] - 0s 2ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 5/100
5/5 [==============================] - 0s 1ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 6/100
5/5 [==============================] - 0s 2ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 7/100
5/5 [==============================] - 0s 6ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 8/100
5/5 [==============================] - 0s 2ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 9/100
5/5 [==============================] - 0s 5ms/sample - loss: 2.9104e-12 - acc: 0.0000e+00
Epoch 10/100
5/5 [==============================] - 0s 1ms/sample - loss: 2.9104e-

## Evaluate the model

In [29]:
x_input=np.array([60,70,80,90])
x_input=x_input.reshape(1,n_seq,n_steps,n_features)
print(x_input.shape)
print(x_input)

(1, 2, 2, 1)
[[[[60]
   [70]]

  [[80]
   [90]]]]


In [30]:
predicted_y_CNN_LSTM=model.predict(x_input)
predicted_y_CNN_LSTM

array([[102.701324]], dtype=float32)

In [31]:
print('inputs',x_input)
print('outputs',predicted_y_CNN_LSTM)

inputs [[[[60]
   [70]]

  [[80]
   [90]]]]
outputs [[102.701324]]


## ConvLSTM
The ConvLSTM was developed for reading two-dimensional spatial-temporal data, but can be adapted for use with univariate time series forecasting.

In [33]:
n_steps=4
X,y=split_sequence(raw_seq,n_steps)
print(X.shape)

n_features =1
n_seq=2
n_steps=2
X_reshaped=X.reshape(X.shape[0],n_seq,1,n_steps,n_features)
print(X_reshaped.shape)
print(X_reshaped)


(5, 4)
(5, 2, 1, 2, 1)
[[[[[10]
    [20]]]


  [[[30]
    [40]]]]



 [[[[20]
    [30]]]


  [[[40]
    [50]]]]



 [[[[30]
    [40]]]


  [[[50]
    [60]]]]



 [[[[40]
    [50]]]


  [[[60]
    [70]]]]



 [[[[50]
    [60]]]


  [[[70]
    [80]]]]]


In [35]:
from tensorflow.keras.layers import ConvLSTM2D

In [36]:
conv_lstm_model=Sequential()
conv_lstm_model.add(ConvLSTM2D(filters=64, kernel_size=(1,2),activation="relu",input_shape=(n_seq,1,n_steps,n_features)))
conv_lstm_model.add(Flatten())
conv_lstm_model.add(Dense(1))

conv_lstm_model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 1, 1, 64)          33536     
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 33,601
Trainable params: 33,601
Non-trainable params: 0
_________________________________________________________________


In [37]:
conv_lstm_model.compile(optimizer="adam",loss='mse')


In [38]:
history=conv_lstm_model.fit(X_reshaped,y,epochs=200)

Epoch 1/200
5/5 [==============================] - 2s 416ms/sample - loss: 5074.1470
Epoch 2/200
5/5 [==============================] - 0s 1ms/sample - loss: 5003.9883
Epoch 3/200
5/5 [==============================] - 0s 2ms/sample - loss: 4930.5562
Epoch 4/200
5/5 [==============================] - 0s 2ms/sample - loss: 4854.9521
Epoch 5/200
5/5 [==============================] - 0s 1ms/sample - loss: 4777.1426
Epoch 6/200
5/5 [==============================] - 0s 2ms/sample - loss: 4697.8301
Epoch 7/200
5/5 [==============================] - 0s 2ms/sample - loss: 4614.2236
Epoch 8/200
5/5 [==============================] - 0s 2ms/sample - loss: 4529.5098
Epoch 9/200
5/5 [==============================] - 0s 2ms/sample - loss: 4443.7939
Epoch 10/200
5/5 [==============================] - 0s 1ms/sample - loss: 4356.8662
Epoch 11/200
5/5 [==============================] - 0s 2ms/sample - loss: 4266.4521
Epoch 12/200
5/5 [==============================] - 0s 2ms/sample - loss: 4173.4277

In [40]:
x_input=np.array([60,70,80,90])
x_input_reshaped=x_input.reshape(1,n_seq,1,n_steps,n_features)
print(x_input_reshaped.shape)
print(x_input_reshaped)

(1, 2, 1, 2, 1)
[[[[[60]
    [70]]]


  [[[80]
    [90]]]]]


In [41]:
predicted_y_ConvLSTM2D=conv_lstm_model.predict(x_input_reshaped)
predicted_y_ConvLSTM2D

array([[104.162926]], dtype=float32)